# Train Encoder

In [16]:
import os
import numpy as np
import json
import argparse

import config
from GPT import GPT
from StimulusModel import LMFeatures
from utils_stim import get_stim
from utils_resp import get_resp
from utils_ridge.ridge import ridge, bootstrap_ridge
np.random.seed(42)

If you get path errors, then change dir using os.chdir to 'whatever-your-root-is/semantic-decoding/

In [19]:
parser = argparse.ArgumentParser()
parser.add_argument("--subject", type = str, required = True)
parser.add_argument("--gpt", type = str, default = "perceived")
parser.add_argument("--sessions", nargs = "+", type = int, default = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 18, 20])
args = parser.parse_args("--subject S1".split())

In [20]:
# training stories
stories = []
with open(os.path.join(config.DATA_TRAIN_DIR, "sess_to_story.json"), "r") as f:
    sess_to_story = json.load(f) 
for sess in args.sessions:
    stories.extend(sess_to_story[str(sess)])

In [5]:
# load gpt
with open(os.path.join(config.DATA_LM_DIR, args.gpt, "vocab.json"), "r") as f:
    gpt_vocab = json.load(f)
gpt = GPT(path = os.path.join(config.DATA_LM_DIR, args.gpt, "model"), vocab = gpt_vocab, device = config.GPT_DEVICE)
features = LMFeatures(model = gpt, layer = config.GPT_LAYER, context_words = config.GPT_WORDS)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
# estimate encoding model
rstim, tr_stats, word_stats = get_stim(stories, features)

In [27]:
rresp = get_resp(args.subject, stories, stack = True)

In [28]:
rresp.shape

(27449, 81126)

In [35]:
nchunks = int(np.ceil(rresp.shape[0] / 5 / config.CHUNKLEN))

In [40]:
weights, alphas, bscorrs = bootstrap_ridge(rstim, rresp, use_corr = False, alphas = config.ALPHAS, nboots = config.NBOOTS, chunklen = config.CHUNKLEN, nchunks = nchunks)

In [49]:
weights.shape

(3072, 81126)

In [47]:
np.save('/workspace/weights/S1.npy', weights)

In [50]:
bscorrs = bscorrs.mean(2).max(0)
vox = np.sort(np.argsort(bscorrs)[-config.VOXELS:])

In [51]:
stim_dict = {story : get_stim([story], features, tr_stats = tr_stats) for story in stories}
resp_dict = get_resp(args.subject, stories, stack = False, vox = vox)
noise_model = np.zeros([len(vox), len(vox)])

In [52]:
## 138min 16.9s

for hstory in stories:
    tstim, hstim = np.vstack([stim_dict[tstory] for tstory in stories if tstory != hstory]), stim_dict[hstory]
    tresp, hresp = np.vstack([resp_dict[tstory] for tstory in stories if tstory != hstory]), resp_dict[hstory]
    bs_weights = ridge(tstim, tresp, alphas[vox])
    resids = hresp - hstim.dot(bs_weights)
    bs_noise_model = resids.T.dot(resids)
    noise_model += bs_noise_model / np.diag(bs_noise_model).mean() / len(stories)

In [53]:
# save
save_location = os.path.join(config.MODEL_DIR, args.subject)
os.makedirs(save_location, exist_ok = True)
np.savez(os.path.join(save_location, "encoding_model_%s" % args.gpt), 
    weights = weights, noise_model = noise_model, alphas = alphas, voxels = vox, stories = stories,
    tr_stats = np.array(tr_stats), word_stats = np.array(word_stats))

# Train Wordrate

In [21]:
import os
import numpy as np
import json
import argparse

import config
from utils_stim import get_story_wordseqs
from utils_resp import get_resp
from utils_ridge.DataSequence import DataSequence
from utils_ridge.util import make_delayed
from utils_ridge.ridge import bootstrap_ridge
np.random.seed(42)

In [23]:
parser = argparse.ArgumentParser()
parser.add_argument("--subject", type = str, required = True)
parser.add_argument("--sessions", nargs = "+", type = int, 
    default = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 18, 20])
args = parser.parse_args("--subject S1".split())

In [25]:
# training stories
stories = []
with open(os.path.join(config.DATA_TRAIN_DIR, "sess_to_story.json"), "r") as f:
    sess_to_story = json.load(f) 
for sess in args.sessions:
    stories.extend(sess_to_story[str(sess)])

In [26]:
# ROI voxels
with open(os.path.join(config.DATA_TRAIN_DIR, "ROIs", "%s.json" % args.subject), "r") as f:
    vox = json.load(f)
        
# estimate word rate model
save_location = os.path.join(config.MODEL_DIR, args.subject)
os.makedirs(save_location, exist_ok = True)

In [27]:
wordseqs = get_story_wordseqs(stories)
rates = {}
for story in stories:
    ds = wordseqs[story]
    words = DataSequence(np.ones(len(ds.data_times)), ds.split_inds, ds.data_times, ds.tr_times)
    rates[story] = words.chunksums("lanczos", window = 3)
nz_rate = np.concatenate([rates[story][5+config.TRIM:-config.TRIM] for story in stories], axis = 0)
nz_rate = np.nan_to_num(nz_rate).reshape([-1, 1])
mean_rate = np.mean(nz_rate)
rate = nz_rate - mean_rate

In [28]:
## 35m 25.9s

for roi in ["speech", "auditory"]:
    resp = get_resp(args.subject, stories, stack = True, vox = vox[roi])
    delresp = make_delayed(resp, config.RESP_DELAYS)
    nchunks = int(np.ceil(delresp.shape[0] / 5 / config.CHUNKLEN))    
    weights, _, _ = bootstrap_ridge(delresp, rate, use_corr = False,
        alphas = config.ALPHAS, nboots = config.NBOOTS, chunklen = config.CHUNKLEN, nchunks = nchunks)
    np.savez(os.path.join(save_location, "word_rate_model_%s" % roi), 
        weights = weights, mean_rate = mean_rate, voxels = vox[roi])

# Run Decoder

In [1]:
import os
import numpy as np
import json
import argparse
import h5py
from pathlib import Path

import config
from GPT import GPT
from Decoder import Decoder, Hypothesis
from LanguageModel import LanguageModel
from EncodingModel import EncodingModel
from StimulusModel import StimulusModel, get_lanczos_mat, affected_trs, LMFeatures
from utils_stim import predict_word_rate, predict_word_times

/chronos_data/rrao/conda/envs/thought2text/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/chronos_data/rrao/conda/envs/thought2text/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/chronos_data/rrao/conda/envs/thought2text/lib/python3.10/site-packages/torch/__init__.py:747: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /opt/conda/conda-bld/pytorch_1712608839953/work/torch/csrc/tensor/python_tensor.cpp:431.)
  _C._set_default_tensor_type(t)


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("--subject", type = str, default = "S1")
parser.add_argument("--experiment", type = str, default = "perceived_speech")
parser.add_argument("--task", type = str, default = "wheretheressmoke")
parser.add_argument("--variant", type = str, default = "base", choices = ["base", "mlp", "gpt2"])
parser.add_argument("--mlp_path", type = str, default = "", help = "Specify path to checkpoint file if `mlp` variant is selected")
args = parser.parse_args("--subject S1 --experiment perceived_speech --task wheretheressmoke".split())

In [3]:
# determine GPT checkpoint based on experiment
if args.experiment in ["imagined_speech"]: gpt_checkpoint = "imagined"
else: gpt_checkpoint = "perceived"

# determine word rate model voxels based on experiment
if args.experiment in ["imagined_speech", "perceived_movies"]: word_rate_voxels = "speech"
else: word_rate_voxels = "auditory"

In [4]:
hf = h5py.File(os.path.join(config.DATA_TEST_DIR, "test_response", args.subject, args.experiment, args.task + ".hf5"), "r")
resp = np.nan_to_num(hf["data"][:])
hf.close()

In [5]:
resp.shape

(291, 81126)

In [6]:
# load gpt
with open(os.path.join(config.DATA_LM_DIR, gpt_checkpoint, "vocab.json"), "r") as f:
    gpt_vocab = json.load(f)
with open(os.path.join(config.DATA_LM_DIR, "decoder_vocab.json"), "r") as f:
    decoder_vocab = json.load(f)
gpt = GPT(path = os.path.join(config.DATA_LM_DIR, gpt_checkpoint, "model"), vocab = gpt_vocab, device = config.GPT_DEVICE)
features = LMFeatures(model = gpt, layer = config.GPT_LAYER, context_words = config.GPT_WORDS)
lm = LanguageModel(gpt, decoder_vocab, nuc_mass = config.LM_MASS, nuc_ratio = config.LM_RATIO)

In [8]:
# load models
load_location = os.path.join(config.MODEL_DIR, args.subject)
word_rate_model = np.load(os.path.join(load_location, "word_rate_model_%s.npz" % word_rate_voxels), allow_pickle = True)
encoding_model = np.load(os.path.join(load_location, "encoding_model_%s.npz" % gpt_checkpoint))
weights = encoding_model["weights"]
noise_model = encoding_model["noise_model"]
tr_stats = encoding_model["tr_stats"]
word_stats = encoding_model["word_stats"]
if args.variant == "base":
    em = EncodingModel(resp, weights, encoding_model["voxels"], noise_model, device = config.EM_DEVICE)
elif args.variant == "mlp":
    em = EncodingModel(resp, weights, encoding_model["voxels"], noise_model, mlp_path = args.mlp_path, device = config.EM_DEVICE)
em.set_shrinkage(config.NM_ALPHA)
assert args.task not in encoding_model["stories"]

In [9]:
print(np.min(encoding_model["voxels"]))
print(np.max(encoding_model["voxels"]))

1186
81012


In [10]:
# predict word times
word_rate = predict_word_rate(resp, word_rate_model["weights"], word_rate_model["voxels"], word_rate_model["mean_rate"])
if args.experiment == "perceived_speech": word_times, tr_times = predict_word_times(word_rate, resp, starttime = -10)
else: word_times, tr_times = predict_word_times(word_rate, resp, starttime = 0)
lanczos_mat = get_lanczos_mat(word_times, tr_times)
len(word_times)

/chronos_data/rrao/Thought2Text/semantic-decoding/decoding/utils_ridge/interpdata.py:159: RuntimeWarning: invalid value encountered in divide
  val = window * np.sin(np.pi*t) * np.sin(np.pi*t/window) / (np.pi**2 * t**2)


1589

In [11]:
# 169m 45.9s

# decode responses
decoder = Decoder(word_times, config.WIDTH)
sm = StimulusModel(lanczos_mat, tr_stats, word_stats[0], device = config.SM_DEVICE)
for sample_index in range(len(word_times)):
    trs = affected_trs(decoder.first_difference(), sample_index, lanczos_mat)
    ncontext = decoder.time_window(sample_index, config.LM_TIME, floor = 5)
    beam_nucs = lm.beam_propose(decoder.beam, ncontext)
    for c, (hyp, nextensions) in enumerate(decoder.get_hypotheses()):
        nuc, logprobs = beam_nucs[c]
        if len(nuc) < 1: continue
        extend_words = [hyp.words + [x] for x in nuc]
        extend_embs = list(features.extend(extend_words))
        stim = sm.make_variants(sample_index, hyp.embs, extend_embs, trs)
        likelihoods = em.prs(stim, trs)
        print('likelihoods.shape: ', likelihoods.shape)
        local_extensions = [Hypothesis(parent = hyp, extension = x) for x in zip(nuc, logprobs, extend_embs)]
        decoder.add_extensions(local_extensions, likelihoods, nextensions)
    decoder.extend(verbose = False)
    break

torch.Size([6, 6, 10000])
torch.Size([6, 10000])
likelihoods.shape:  (6,)


In [41]:
if args.experiment in ["perceived_movie", "perceived_multispeaker"]: decoder.word_times += 10
save_location = os.path.join(config.RESULT_DIR, args.subject, args.experiment)
os.makedirs(save_location, exist_ok = True)
decoder.save(os.path.join(save_location, args.task))

# Evaluate outputs

In [ ]:
import os
import numpy as np
import json
import argparse

import config
from utils_eval import generate_null, load_transcript, windows, segment_data, WER, BLEU, METEOR, BERTSCORE

In [5]:
parser = argparse.ArgumentParser()
parser.add_argument("--subject", type = str, required = True)
parser.add_argument("--experiment", type = str, required = True)
parser.add_argument("--task", type = str, required = True)
parser.add_argument("--metrics", nargs = "+", type = str, default = ["WER", "BLEU", "METEOR", "BERT"])
parser.add_argument("--references", nargs = "+", type = str, default = [])
parser.add_argument("--null", type = int, default = 10)
args = parser.parse_args("--subject S1 --experiment perceived_speech --task wheretheressmoke".split())

In [6]:
if len(args.references) == 0:
    args.references.append(args.task)
    
with open(os.path.join(config.DATA_TEST_DIR, "eval_segments.json"), "r") as f:
    eval_segments = json.load(f)

In [ ]:
# 2m 52.1s

# load language similarity metrics
metrics = {}
if "WER" in args.metrics: metrics["WER"] = WER(use_score = True)
if "BLEU" in args.metrics: metrics["BLEU"] = BLEU(n = 1)
if "METEOR" in args.metrics: metrics["METEOR"] = METEOR()
if "BERT" in args.metrics: metrics["BERT"] = BERTSCORE(
    idf_sents = np.load(os.path.join(config.DATA_TEST_DIR, "idf_segments.npy")), 
    rescale = False, 
    score = "recall")

In [10]:
# load prediction transcript
pred_path = os.path.join(config.RESULT_DIR, args.subject, args.experiment, args.task + ".npz")
pred_data = np.load(pred_path)
pred_words, pred_times = pred_data["words"], pred_data["times"]

In [11]:
# 8m 8.9sec

# generate null sequences
if args.experiment in ["imagined_speech"]: gpt_checkpoint = "imagined"
else: gpt_checkpoint = "perceived"
null_word_list = generate_null(pred_times, gpt_checkpoint, args.null)

In [20]:
window_scores, window_zscores = {}, {}
story_scores, story_zscores = {}, {}
for reference in args.references:

    # load reference transcript
    ref_data = load_transcript(args.experiment, reference)
    ref_words, ref_times = ref_data["words"], ref_data["times"]

    # segment prediction and reference words into windows
    window_cutoffs = windows(*eval_segments[args.task], config.WINDOW)
    ref_windows = segment_data(ref_words, ref_times, window_cutoffs)
    pred_windows = segment_data(pred_words, pred_times, window_cutoffs)
    null_window_list = [segment_data(null_words, pred_times, window_cutoffs) for null_words in null_word_list]
    
    for mname, metric in metrics.items():

        # get null score for each window and the entire story
        window_null_scores = np.array([metric.score(ref = ref_windows, pred = null_windows) 
                                        for null_windows in null_window_list])
        story_null_scores = window_null_scores.mean(1)

        # get raw score and normalized score for each window
        window_scores[(reference, mname)] = metric.score(ref = ref_windows, pred = pred_windows)
        window_zscores[(reference, mname)] = (window_scores[(reference, mname)] 
                                                - window_null_scores.mean(0)) / window_null_scores.std(0)

        # get raw score and normalized score for the entire story
        story_scores[(reference, mname)] = metric.score(ref = ref_windows, pred = pred_windows)
        story_zscores[(reference, mname)] = (story_scores[(reference, mname)].mean()
                                                - story_null_scores.mean()) / story_null_scores.std()

save_location = os.path.join(config.REPO_DIR, "scores", args.subject, args.experiment)
os.makedirs(save_location, exist_ok = True)
np.savez(os.path.join(save_location, args.task), 
            window_scores = window_scores, window_zscores = window_zscores, 
            story_scores = story_scores, story_zscores = story_zscores)

ValueError: After applying the transforms on the reference and hypothesis sentences, their lengths must match. Instead got 49 reference and 56 hypothesis sentences.

In [11]:
import numpy as np
predictions = np.load('../../results/S1/perceived_speech/wheretheressmoke_BASE_MLP.npz', allow_pickle=True)
print(' '.join(predictions['words']))

she was doing i don't remember her being drunk but i was sure i saw her leave with my son at this point she is talking about the car accident i remember thinking she must be in pain i thought of her as my best friend but she had been shot i told myself she was dead i got the idea that it wasn't her but i was wrong she had her life but the people around me did not it was a time where she felt alone i am sorry but she needed to make it through it it is her family and not me that are suffering now i think she was in pain and the idea of my being there made her sick i also know that the doctor was there because of me and he is a good guy so i went on to do everything right and i didn't go home but he got to see my work and he liked the results i made the decision to hire someone for his project and i did the work myself the company did it and the guy who ran it was the manager at the time and i think they really liked the music so the other kids are just sitting around watching it i think 

In [ ]:
grids = load_textgrids(stories, config.DATA_TRAIN_DIR)
with open(os.path.join(config.DATA_TRAIN_DIR, "respdict.json"), "r") as f:
    respdict = json.load(f)
trfiles = load_simulated_trfiles(respdict)
wordseqs = make_word_ds(grids, trfiles)
return wordseqs

In [2]:
import numpy as np

base = np.load('../scores/S1/perceived_speech/wheretheressmoke.npz', allow_pickle=True)
mlp = np.load('../scores/S1/perceived_speech/wheretheressmoke_mlp.npz', allow_pickle=True)
gpt2 = np.load('../scores/S1/perceived_speech/wheretheressmoke_gpt2.npz', allow_pickle=True)

In [4]:
print(base['story_zscores'])
print(mlp['story_zscores'])
print(gpt2['story_zscores'])

{('wheretheressmoke', 'WER'): 12.656749219799638, ('wheretheressmoke', 'BLEU'): 5.750169501152248, ('wheretheressmoke', 'METEOR'): 6.447097391696413, ('wheretheressmoke', 'BERT'): 12.795977}
{('wheretheressmoke', 'WER'): 1.9219846660381588, ('wheretheressmoke', 'BLEU'): -0.5513700296316832, ('wheretheressmoke', 'METEOR'): 1.2374410749343323, ('wheretheressmoke', 'BERT'): 2.7528594}
{('wheretheressmoke', 'WER'): -13.071730618874115, ('wheretheressmoke', 'BLEU'): -22.25487294995505, ('wheretheressmoke', 'METEOR'): -16.06553233150744, ('wheretheressmoke', 'BERT'): -33.929707}


In [ ]:
base.close()
mlp.close()
gpt2.close()

In [ ]:
# Access the contents of the NPZ file
# The contents are stored as a dictionary-like object
# You can access each array using its key
array1 = data['array1_key']
array2 = data['array2_key']

In [ ]:
# Close the NPZ file after use
data.close()